In [27]:
import pickle
import torch
import numpy

# torch.serialization.add_safe_globals([numpy._core.multiarray._reconstruct, numpy.ndarray, numpy.dtype, numpy])

import pandas as pd
from torch_geometric.loader import DataLoader

# with open("data/raw/epl_2015.pkl", "rb") as f:
with open("../data/raw/epl_2020.pkl", "rb") as f:
    loaded_list = pickle.load(f)

print(len(loaded_list))
# print(loaded_list[0])
events: pd.DataFrame = loaded_list[0]["events"]

events

380


game_id  \
league             season game                                             
ENG-Premier League 2021   2020-09-12 Crystal Palace-Southampton  1485186   
                          2020-09-12 Crystal Palace-Southampton  1485186   
                          2020-09-12 Crystal Palace-Southampton  1485186   
                          2020-09-12 Crystal Palace-Southampton  1485186   
                          2020-09-12 Crystal Palace-Southampton  1485186   
...                                                                  ...   
                          2020-09-12 Crystal Palace-Southampton  1485186   
                          2020-09-12 Crystal Palace-Southampton  1485186   
                          2020-09-12 Crystal Palace-Southampton  1485186   
                          2020-09-12 Crystal Palace-Southampton  1485186   
                          2020-09-12 Crystal Palace-Southampton  1485186   

                                                                     period  \
league             season game                                                
ENG-Premier League 2021   2020-09-12 Crystal Palace-Southampton   FirstHalf   
                          2020-09-12 Crystal Palace-Southampton   FirstHalf   
                          2020-09-12 Crystal Palace-Southampton   FirstHalf   
                          2020-09-12 Crystal Palace-Southampton   FirstHalf   
                          2020-09-12 Crystal Palace-Southampton   FirstHalf   
...                                                                     ...   
                          2020-09-12 Crystal Palace-Southampton  SecondHalf   
                          2020-09-12 Crystal Palace-Southampton    PostGame   
                          2020-09-12 Crystal Palace-Southampton    PostGame   
                          2020-09-12 Crystal Palace-Southampton    PreMatch   
                          2020-09-12 Crystal Palace-Southampton    PreMatch   

                                                                 minute  \
league             season game                                            
ENG-Premier League 2021   2020-09-12 Crystal Palace-Southampton       0   
                          2020-09-12 Crystal Palace-Southampton       0   
                          2020-09-12 Crystal Palace-Southampton       0   
                          2020-09-12 Crystal Palace-Southampton       0   
                          2020-09-12 Crystal Palace-Southampton       0   
...                                                                 ...   
                          2020-09-12 Crystal Palace-Southampton      96   
                          2020-09-12 Crystal Palace-Southampton       0   
                          2020-09-12 Crystal Palace-Southampton       0   
                          2020-09-12 Crystal Palace-Southampton       0   
                          2020-09-12 Crystal Palace-Southampton       0   

                                                                 second  \
league             season game                                            
ENG-Premier League 2021   2020-09-12 Crystal Palace-Southampton     0.0   
                          2020-09-12 Crystal Palace-Southampton     0.0   
                          2020-09-12 Crystal Palace-Southampton     0.0   
                          2020-09-12 Crystal Palace-Southampton     2.0   
                          2020-09-12 Crystal Palace-Southampton     5.0   
...                                                                 ...   
                          2020-09-12 Crystal Palace-Southampton     2.0   
                          2020-09-12 Crystal Palace-Southampton     0.0   
                          2020-09-12 Crystal Palace-Southampton     0.0   
                          2020-09-12 Crystal Palace-Southampton     0.0   
                          2020-09-12 Crystal Palace-Southampton     0.0   

                                                                 expanded_minute  \
league      

In [41]:
tensor = torch.load("../data/processed/data_1000.pt")
print(tensor)

Data(home_x=[11, 1], home_edge_index=[2, 70], home_edge_weight=[70], away_x=[11, 1], away_edge_index=[2, 49], away_edge_weight=[49], start_minute=0, end_minute=30, home_players=[11], away_players=[11], season='epl_2015', match_id=959656, home_team_name='Bournemouth', away_team_name='Stoke', data_id=1000)


/tmp/ipykernel_626685/3410397509.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tensor = torch.load('../data/processed/data_1000.pt')


In [45]:
# from src.our_dataloader import SequentialSoccerDataset, CumulativeSoccerDataset
from src.dataloader_prova2 import SequentialSoccerDataset, CumulativeSoccerDataset

# torch.serialization.add_safe_globals([numpy._core.multiarray._reconstruct, numpy.ndarray, numpy.dtype, numpy])

# dataset = SequentialSoccerDataset(root="data", ending_year=2016)
dataset = SequentialSoccerDataset(root="../data", ending_year=2016)
dataloader = DataLoader(dataset=dataset, batch_size=32, shuffle=True, num_workers=0)

# cum_data = CumulativeSoccerDataset(root="data", ending_year=2016, time_interval=20)
cum_data = CumulativeSoccerDataset(root="../data", ending_year=2016, time_interval=20)
dataloader2 = DataLoader(dataset=cum_data, batch_size=32, shuffle=True, num_workers=0)

In [46]:
iterator = iter(dataloader2)
first = iterator._next_data()
first

/home/oriolmonge/src/Erasmus/Graph ML/tudelft-graph-ml-football-predictions/src/dataloader_prova2.py:204: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(Pat

RuntimeError: Could not infer dtype of NoneType

In [49]:
for i in range(10):
    print(cum_data[i])

Data(home_x=[11, 1], home_edge_index=[2, 70], home_edge_weight=[70], away_x=[11, 1], away_edge_index=[2, 66], away_edge_weight=[66], start_minute=0, end_minute=30, home_players=[11], away_players=[11], season='epl_2015', match_id=958427, home_team_name='Bournemouth', away_team_name='Aston Villa', data_id=0)
Data(home_x=[12, 1], home_edge_index=[2, 65], home_edge_weight=[65], away_x=[12, 1], away_edge_index=[2, 69], away_edge_weight=[69], start_minute=30, end_minute=60, home_players=[12], away_players=[12], season='epl_2015', match_id=958427, home_team_name='Bournemouth', away_team_name='Aston Villa', data_id=1)
Data(home_x=[13, 1], home_edge_index=[2, 81], home_edge_weight=[81], away_x=[13, 1], away_edge_index=[2, 57], away_edge_weight=[57], start_minute=60, end_minute=90, home_players=[13], away_players=[13], season='epl_2015', match_id=958427, home_team_name='Bournemouth', away_team_name='Aston Villa', data_id=2)
Data(home_x=[10, 1], home_edge_index=[2, 17], home_edge_weight=[17], aw

/home/oriolmonge/src/Erasmus/Graph ML/tudelft-graph-ml-football-predictions/src/dataloader_prova2.py:204: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(Pat

In [ ]:
import torch
from src.rnn import SimpleRNNModel  # your RNN model class

# Load model
input_size = 4
hidden_size = 64
num_layers = 1
output_size = 3

device = torch.device("cpu")  # or "cuda" if available
model = SimpleRNNModel(input_size, hidden_size, num_layers, output_size).to(device)

# Load the saved weights
model.load_state_dict(torch.load("rnn_soccer_model.pt", map_location=device))
model.eval()

# Print all parameter names and shapes
for name, param in model.named_parameters():
    print(name, param.shape)

print(model.fc.weight)
print(model.rnn.weight_ih_l0)

w = model.rnn.weight_ih_l0
print("mean:", w.mean().item(), "std:", w.std().item())


rnn.weight_ih_l0 torch.Size([64, 7])
rnn.weight_hh_l0 torch.Size([64, 64])
rnn.bias_ih_l0 torch.Size([64])
rnn.bias_hh_l0 torch.Size([64])
fc.weight torch.Size([3, 64])
fc.bias torch.Size([3])
Parameter containing:
tensor([[ 0.4346,  0.1026,  0.4516,  0.3839,  0.1285,  0.2090,  0.3302, -0.1827,
          0.0467, -0.1508, -0.1055, -0.1939, -0.4593,  0.1966,  0.1498,  0.2530,
          0.2244, -0.2619,  0.0102,  0.4832,  0.6160,  0.2769, -0.3454, -0.6119,
         -0.2597,  0.1080, -0.0425, -0.4147,  0.2931,  0.0525, -0.0240,  0.2655,
          0.1518, -0.0439,  0.1812,  0.1796,  0.0733,  0.4851,  0.3054, -0.1243,
         -0.1087, -0.1135,  0.2187,  0.0353, -0.4593,  0.0885,  0.3266, -0.1325,
          0.1439, -0.1189,  0.0468,  0.0751,  0.0705,  0.2635, -0.3702, -0.2868,
         -0.3040,  0.0509, -0.3943,  0.2524,  0.1093,  0.0100,  0.1455,  0.5376],
        [ 0.0215, -0.0910,  0.0755,  0.0399, -0.0662,  0.0679, -0.0641, -0.0473,
         -0.0613, -0.2377,  0.0745,  0.2546,  0.0227,  

In [13]:
import torch

data_file = "data/processed_sequential/data_14509.pt"
data = torch.load(data_file, weights_only=False)

# Print the object
print(data)

# Inspect individual parts
print("Home graph node features:", data.home_x.shape)
print("Away graph node features:", data.away_x.shape)
print("Home edges:", data.home_edge_index.shape)
print("Away edges:", data.away_edge_index.shape)
print("Final result (one-hot):", data.final_result)
print("Current goals:", data.current_home_goals, data.current_away_goals)
print("Start-End minute:", data.start_minute.item(), data.end_minute.item())

# List all attributes
print("All keys:", list(data.keys()))


Data(home_x=[11, 1], home_edge_index=[2, 14], home_edge_weight=[14], home_players=[11], away_x=[1, 1], away_edge_index=[2, 0], away_edge_weight=[0], away_players=[0], start_minute=85, end_minute=90, current_home_goals=0, current_away_goals=0, final_home_goals=0, final_away_goals=0, final_result=[3], home_team='Watford', away_team='Manchester City', season='epl_2016', match_id=1080525, data_id=14509)
Home graph node features: torch.Size([11, 1])
Away graph node features: torch.Size([1, 1])
Home edges: torch.Size([2, 14])
Away edges: torch.Size([2, 0])
Final result (one-hot): tensor([0., 1., 0.])
Current goals: 0 0
Start-End minute: 85 90
All keys: ['home_edge_index', 'away_team', 'current_home_goals', 'final_home_goals', 'start_minute', 'end_minute', 'home_players', 'season', 'away_x', 'final_result', 'away_edge_index', 'home_edge_weight', 'away_players', 'home_team', 'final_away_goals', 'data_id', 'current_away_goals', 'home_x', 'away_edge_weight', 'match_id']
